In [1]:
#Import them packages
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import mysql.connector

In [2]:
#MySql connector for accessing data from your database on the go.

try:
    cnx = mysql.connector.connect(user='*****',password='*****',host='127.0.0.1',database='stackoverflow_stackexchange')
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)


In [3]:
#Imported one million answers from Posts table/file.
post_df= pd.DataFrame()
query = ('select Score,Body,Tags from Posts where PostTypeId=2 limit 1000000')
post_df=post_df.append(pd.read_sql(query, cnx))
scoreTrain_label=post_df.Score.as_matrix()

In [4]:
post_df

,Score,Body,Tags
0,329,<p>An explicit cast to double isn't necessary....,None
1,296,"<p>Well, here's how we do it on Stack Overflow...",None
2,37,<p>For a table like this:</p>&#xA;&#xA;<pre><c...,None
3,26,"<p>Many years ago, to provide an <a href=""http...",None
4,20,<p>The best way that I know of because of leap...,None
5,94,<p>The answer by phpguy is correct but I think...,None
6,27,<p>@jeff</p>&#xA;&#xA;<p>IMHO yours seems a li...,None
7,54,<p>There are no HTTP headers that will report ...,None
8,25,<p>I've had no trouble using the BSD sockets A...,None
9,105,<p>The parent <code>div</code> needs to have a...,None


In [5]:
##Converting to classification data. Assigning range to scores.

post_df.loc[post_df.Score>999,'Score']=1
post_df.loc[(post_df.Score>499)&(post_df.Score<=999), 'Score']=2
post_df.loc[(post_df.Score>0)& (post_df.Score<=499), 'Score']=3
post_df.loc[post_df.Score<=0, 'Score']=4

In [6]:
#Tf-Idf Vectorizer to calculate word-vectors.

from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(stop_words='english')
vectorized_data = v.fit_transform(post_df.Body)

In [7]:
#Initialize all models.
ada_clf=AdaBoostClassifier(n_estimators=100)
rf_clf=RandomForestClassifier()
mlp_clf=MLPClassifier()

In [8]:
#Using SVD to convert to Dense Matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
svd = TruncatedSVD(n_components=300, n_iter=7)
normalizer = Normalizer(copy=False)
lsa = sk.pipeline.make_pipeline(svd, normalizer)
d_reduced = lsa.fit_transform(vectorized_data)

In [9]:
#Converting test data into suitable form.Test data is a completely different set from the training data. Stored separately.
#It contains 200000 answers.
test_df = pd.read_csv('/home/cloakedprat/csv1/testData2.csv')
test_data = v.transform(test_df['Body'])
lsa2 = sk.pipeline.make_pipeline(svd, normalizer)
d_reduced_testData = lsa2.fit_transform(test_data)

In [10]:
##Converting to classification data
test_df.loc[test_df.Score>999,'Score']=1
test_df.loc[(test_df.Score>499)&(test_df.Score<=999), 'Score']=2
test_df.loc[(test_df.Score>0)& (test_df.Score<=499), 'Score']=3
test_df.loc[test_df.Score<=0, 'Score']=4

In [13]:
#Multi-layer perceptron.
mlp_clf.fit(d_reduced,scoreTrain_label)
mlp_predict=mlp_clf.predict(d_reduced_testData)

In [15]:
print(sk.metrics.accuracy_score(mlp_predict,test_df.Score))

0.68636


In [11]:
#Random-forest___WTC!!
rf_clf.fit(d_reduced,scoreTrain_label)
predict=rf_clf.predict(d_reduced_testData)
print(sk.metrics.accuracy_score(predict,test_df.Score))

0.92766
